# Prelim

This notebook explains how to handle filtering and analysis when multiple samples need to be analyzed together. We will use the multiome data of the T-cell depleted bone marrow in the [SEACells](https://www.google.com/search?q=seacells+biorxiv&oq=seacells+biorxi&aqs=chrome.0.0i512j69i57.3474j0j7&sourceid=chrome&ie=UTF-8) manuscript. The data was generated using two 10X channels.

The initial part of the notebook shows how to handle multiple samples. After combinig the two samples, the notebook follows the same flow of the multiome single-cell sample notebook. 

Data available at `s3://fh-pi-setty-m-eco-public/single-cell-primers/multi-sample-multiome/`

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import scanpy as sc
import pyranges as pr
import warnings

In [3]:
import palantir 
import phenograph
import harmony

findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


In [4]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
%matplotlib inline

sns.set_style('ticks')
matplotlib.rcParams['figure.figsize'] = [4, 4]
matplotlib.rcParams['figure.dpi'] = 100
matplotlib.rcParams['image.cmap'] = 'Spectral_r'
warnings.filterwarnings(action="ignore", module="matplotlib", message="findfont")

# Utility functions

In [6]:
def log_transform(ad, ps=0.1):
    ad.X.data = np.log2(ad.X.data + ps) - np.log2(ps)

In [7]:
from joblib import Parallel, delayed
from scipy.sparse import hstack, csr_matrix, issparse
import gc


def _dot_func(x, y):
    return x.dot(y)


def impute_data(dm_res, ad, n_steps=3):
    T_steps = dm_res['T'] ** n_steps
    T_steps = T_steps.astype(np.float32)

    # RUn in parallel
    seq = np.append(np.arange(0, ad.X.shape[1], 100), [ad.X.shape[1]])
    res = Parallel(n_jobs=-1)(delayed(_dot_func)(T_steps, ad.X[:, seq[i - 1]:seq[i]]) for i in range(1, len(seq)))
    imputed_data = hstack(res)
    imputed_data = imputed_data.todense()
    imputed_data[imputed_data < 1e-2] = 0
    gc.collect()

    return imputed_data

# Load data

In [8]:
# Copy over data to the current working directory
<<<<<<< local
!aws s3 sync s3://fh-pi-setty-m-eco-public/single-cell-primers/multi-sample-multiome/ ../data/multi-sample-multiome/
=======
!aws s3 sync s3://fh-pi-setty-m-eco-public/single-cell-primers/multi-sample-multiome/ multi-sample-multiome/
>>>>>>> remote

In [9]:
<<<<<<< local
data_dir = '../data/multi-sample-multiome/'
=======
data_dir = 'multi-sample-multiome/'
>>>>>>> remote

In [10]:
# RNA and ATAC data
samples = ['rep1', 'rep2']

rna_ad_dict, atac_ad_dict = dict(), dict()
for r in samples:
    comb_ad = sc.read_10x_h5(data_dir + r + '/filtered_feature_bc_matrix.h5', gex_only=False)
    # Use a name which is indicative of the data
    comb_ad.obs_names = 'bm_multiome_' + r + '#'+ comb_ad.obs_names
<<<<<<< local
    comb_ad.uns['features'] = pd.read_csv(data_dir + "features.tsv.gz")
=======
    
>>>>>>> remote
    # RNA
    rna_ad_dict[r] = comb_ad[:, comb_ad.var['feature_types'] == 'Gene Expression']
    rna_ad_dict[r].var_names_make_unique()
    rna_ad_dict[r].obs['sample'] = r
    
    # ATAC
    atac_ad_dict[r] = comb_ad[:, comb_ad.var['feature_types'] == 'Peaks']
    atac_ad_dict[r].var_names_make_unique()


In [11]:
# Cells from each sample 
sample_cells = dict()
for sample in samples:
    sample_cells[sample] = rna_ad_dict[sample].obs_names

In [12]:
# Per barcode metrics 
bc_metrics_dict = dict()
for sample in samples:
    bc_metrics_dict[sample] = pd.read_csv(data_dir + r + '/per_barcode_metrics.csv', index_col=0)
    bc_metrics_dict[sample].index = 'bm_multiome_' + sample + '#' +bc_metrics_dict[sample].index 
    # FRIP
    bc_metrics_dict[sample]['FRIP'] = bc_metrics_dict[sample]['atac_peak_region_fragments']/bc_metrics_dict[sample]['atac_fragments']
    
    # Update subset of cells
    sample_cells[sample] = sample_cells[sample].intersection(bc_metrics_dict[sample].index)


# Preprocess

## QC

In [13]:
# QC metrics include mitochondrial fractions from scanpy 
for r in samples:
    rna_ad = rna_ad_dict[r]
    rna_ad.var['mt'] = rna_ad.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(rna_ad, qc_vars=['mt'], inplace=True, percent_top=None, log1p=False, )

In [14]:
# QC metrics include mitochondrial fractions from scanpy 
for r in samples:
    atac_ad = atac_ad_dict[r]
    sc.pp.calculate_qc_metrics(atac_ad, inplace=True, percent_top=None, log1p=False, )

### Filtering cells based on RNA

In [15]:
fig = palantir.plot.FigureGrid(len(samples), 5)
for sample, ax in zip (samples, fig):
    rna_ad = rna_ad_dict[sample]
    ax.hist(np.log10(rna_ad.obs['total_counts'][sample_cells[sample]]), 50)
    ax.set_title(sample)
sns.despine()
plt.suptitle('log10 molecule counts')

In [16]:
fig = palantir.plot.FigureGrid(len(samples), 5)
for sample, ax in zip (samples, fig):
    rna_ad = rna_ad_dict[sample]
    THRESHOLDS = np.log10(np.percentile((rna_ad.obs['total_counts']), [2.5, 99.5]))
    ax.hist(np.log10(rna_ad.obs['total_counts'][sample_cells[sample]]), 50)
    ax.vlines(THRESHOLDS, ax.get_ylim()[0], ax.get_ylim()[1], color='black', linestyle='--')
    ax.set_title(sample)
sns.despine()
plt.suptitle('log10 molecule counts')

In [17]:
for sample, ax in zip (samples, fig):
    rna_ad = rna_ad_dict[sample]
    cells = sample_cells[sample]
    
    # Filter
    THRESHOLDS = np.percentile((rna_ad.obs['total_counts']), [2.5, 99.5])
    sample_cells[sample] = cells[(rna_ad.obs['total_counts'][cells] > THRESHOLDS[0]) & \
        (rna_ad.obs['total_counts'][cells] < THRESHOLDS[1])]

In [18]:
fig = palantir.plot.FigureGrid(len(samples), 5)
for sample, ax in zip (samples, fig):
    rna_ad = rna_ad_dict[sample]
    ax.hist(np.log10(rna_ad.obs['total_counts'][sample_cells[sample]]), 50)
    ax.set_title(sample)
sns.despine()
plt.suptitle('log10 molecule counts')

### Filtering based on ATAC

In [19]:
## CAUTION: CellRanger peak calling is unreliable and we only use it here to filter cells with low 
##          fraction of reads in peaks. This is ok since CellRanger peak caller dramatically over estimates 
##          the peak widths. We might incur some false negatives, but thats an acceptable error for filtering   
fig = palantir.plot.FigureGrid(len(samples), 5)
for sample, ax in zip (samples, fig):
    ax.hist(bc_metrics_dict[sample]['FRIP'][sample_cells[sample]], 50)
    ax.set_title(sample)
sns.despine()
plt.suptitle('Fraction of reads in peaks')

In [20]:
# Given the uneven distribution, we will skip filtering by ATAC here.
THRESHOLD = 00
for sample in samples:
    sample_cells[sample] = sample_cells[sample][bc_metrics_dict[sample]['FRIP'][sample_cells[sample]] > THRESHOLD]

In [21]:
fig = palantir.plot.FigureGrid(len(samples), 5)
for sample, ax in zip (samples, fig):
    rna_ad = rna_ad_dict[sample]
    ax.hist(np.log10(rna_ad.obs['total_counts'][sample_cells[sample]]), 50)
    ax.set_title(sample)
sns.despine()
plt.suptitle('log10 molecule counts')

### Filter based on mitochondria

In [ ]:
# Mitochondrial fractions compared to total molecules (colored by density)
MITO_THRESHOLD = 30
fig = palantir.plot.FigureGrid(len(samples), 5)
for sample, ax in zip(samples, fig):
    rna_ad = rna_ad_dict[sample]
    
    # Density 
    x,y,dens = palantir.plot.density_2d(rna_ad.obs['total_counts'][sample_cells[sample]], 
               rna_ad.obs['pct_counts_mt'][sample_cells[sample]])
    
    ax.scatter(x, y, c=dens, s=5)
    ax.hlines(MITO_THRESHOLD, ax.get_xlim()[0], ax.get_xlim()[1], color='black', linestyle='--')
    ax.set_title(sample)
sns.despine()



In [ ]:
for sample in samples:
    cells = sample_cells[sample]
    sample_cells[sample] = cells[rna_ad_dict[sample].obs['pct_counts_mt'][cells] < MITO_THRESHOLD]

## Combine samples 

In [ ]:
sample = samples[0]
ad = rna_ad_dict[sample][sample_cells[sample]]
for sample in samples[1:]:
    ad = ad.concatenate(rna_ad_dict[sample][sample_cells[sample]], 
                            index_unique=None, batch_key='batch')
ad

In [ ]:
# Clean up var object
ad.var = pd.DataFrame(index=ad.var_names)

In [ ]:
# Filter cells
sc.pp.filter_genes(ad, min_cells=50)

In [ ]:
ad

In [ ]:
raw_ad = ad.copy()

### Doublet scores  (post filtering)

In [ ]:
# Install scrublet using `pip install scrublet`
import scrublet as scr

In [ ]:
doublet_scores = pd.Series(0.0, index=ad.obs_names)

for sample in samples:
    cells = ad.obs_names[ad.obs['sample'] == sample]
    scrub = scr.Scrublet(raw_ad[cells, :].X)
    scores, predicted_doublets = scrub.scrub_doublets()
    doublet_scores[cells] = scores

ad.obs['DoubletScores'] = doublet_scores.values

## Analysis

In [ ]:
# Normalize and log transform
sc.pp.normalize_per_cell(ad)
log_transform(ad)

In [ ]:
# Highly variable genes [Num. of genes is a paramter, typically 1.5-2.5k genes work well]
sc.pp.highly_variable_genes(ad, flavor='cell_ranger', n_top_genes=1500)
ad

In [ ]:
# PCA
# 50 comps is an approxmiation here - choose number of components by knee point or fraction of explained variance
n_comps = 50
sc.pp.pca(ad, use_highly_variable=True, n_comps=n_comps)

In [ ]:
# UMAP and Leiden - requires the computation of nearest neighbors
# Ignore numba warnings in neighborhood computation
warnings.filterwarnings('ignore')
sc.pp.neighbors(ad, use_rep='X_pca')
warnings.filterwarnings('default')
sc.tl.umap(ad)
sc.tl.leiden(ad)
ad

In [ ]:
# Diffusion maps 
warnings.filterwarnings('ignore')
dm_res = palantir.utils.run_diffusion_maps(pd.DataFrame(ad.obsm['X_pca'], index=ad.obs_names))
warnings.filterwarnings('default')
ad.obsp['DM_Kernel'] = dm_res['kernel']
ad.obsm['DM_EigenVectors'] = dm_res['EigenVectors'].values
ad.uns['DMEigenValues'] = dm_res['EigenValues'].values
ad

In [ ]:
# Force directed layout
ad.obsm['X_FDL'] = harmony.plot.force_directed_layout(dm_res['kernel']).values

In [ ]:
# Imputation
from scipy.sparse import csr_matrix
ad.layers['MAGIC_imputed_data'] = csr_matrix(impute_data(dm_res, ad))

# Visualize results

In [ ]:
ad.obs['log_n_counts'] = np.log10(ad.obs['total_counts'])

Note that in the sample the two samples coincide very clearly indicating that there is no batch effect. This is completely expected since the two replicates were generated using cells from the same individual. Separation of cells based on sample here is a good diagnostic for batch effects

In [ ]:
sc.pl.scatter(ad, basis='umap', color=['log_n_counts', 'DoubletScores', 'sample'], frameon=False)

In [ ]:
sc.pl.scatter(ad, basis='FDL', color=['log_n_counts', 'DoubletScores', 'sample'], frameon=False)

In [ ]:
sc.pl.scatter(ad, basis='umap', color=['leiden'], frameon=False)

In [ ]:
sc.pl.scatter(ad, basis='FDL', color=['leiden'], frameon=False)

# Celltype annotation

In [ ]:
marker_dict = {'HSPC': ['CD34'], 
               'CLP/Bcells': ["CD79B", "EBF1", "PAX5"],
               'B cells': ["CD19"], 
               'Ery': ['GATA1', 'GATA2'], 
               'DC': ['IRF4', 'IRF8'],
               'Mono': ['MPO'],
               'Megakaryocyte': ['ITGA2B'], 
              }

In [ ]:
sc.pl.dotplot(ad, marker_dict, 'leiden', dendrogram=True)

In [ ]:
genes = pd.Series([
    "CD34", "CD38", # HSPC
    "CD79B", "EBF1", "PAX5", # CLP / B cell lineage 
    "CD19", "CD20", # Mature B cell markers
    "GATA1",  # Erythroid lineage
    "IRF8", # DC lineage
    "MPO", # Monocyte lineage
    "ITGA2B", #CD41 - Megakaryocyte,     
])
genes = genes[genes.isin(ad.var_names)]

In [ ]:
sc.pl.embedding(ad, basis='umap', color=genes, 
                layer='MAGIC_imputed_data', frameon=False) 

# Save

In [ ]:
# Attach raw counts
ad.raw = sc.AnnData(raw_ad[ad.obs_names, :][:, ad.var_names].X)

In [ ]:
ad.write(data_dir + 'bm_multiome_rna.h5ad')

In [ ]:
# Export cell names for ATAC - only this subset of cells will be use for ATAC analysis
pd.DataFrame(ad.obs_names).to_csv(data_dir + 'bm_multiome_cells.csv')

In [ ]:

data_dir

In [ ]:
!ls -ltrh $data_dir